On the [wiki page](https://github.com/agjftucker/neva/wiki/Pseudospectral-Solution) we got as far as,
\begin{equation*}
  \left(D_{y_{k}}\boldsymbol{x}\right)=
 -\left(D_{\boldsymbol{x}}\boldsymbol{G}\right)^{-1}\left(D_{y_{k}}\boldsymbol{G}\right)
 \qquad\qquad
 (1)
\end{equation*}
and
\begin{equation*}
 \left(D_{y_{k}y_{l}}\boldsymbol{x}\right) =
 -\left(D_{\boldsymbol{x}}\boldsymbol{G}\right)^{-1}\left(
\left( D_{y_{k}y_{l}}\boldsymbol{G}\right)
+\sum_{n} \left(D_{x_{n}y_{k}}\boldsymbol{G}\right) \left(D_{y_{l}}
  x_{n}\right)
+\sum_{m}\left(D_{x_{m}y_{l}}\boldsymbol{G}\right)\left(D_{y_{k}}x_m\right)
+\sum_{mn}\left(D_{y_{k}}x_{m}\right)\left(D_{x_{m}x_{n}}\boldsymbol{G}\right) \left(D_{y_{l}}x_{n}\right)
\right).
 \qquad(2)
\end{equation*}

But let's change the suggestion in its final paragraph to
\begin{equation*}
  G_{j}(\boldsymbol{x}, \boldsymbol{y}) = \frac{\left(1+x_{j}\right)\eta_{j}}{1-x_{j}} +
  \log\!\left(z_{j}-\sum_{i\neq j}v^{(j)}_{i} L_{ij}\right) - y_{j}
\end{equation*}
so that the associated mapping is from $x\in[-1,1]$ to $y$ in the appropriate semi-infinite interval,
\begin{equation*}
  G_{j}(\boldsymbol{x}, \boldsymbol{y})=0\quad\implies\quad
  y_{j} = \frac{\left(1+x_{j}\right)\eta_{j}}{1-x_{j}} +
  \log\!\left(z_{j}-\sum_{i\neq j}v^{(j)}_{i} L_{ij}\right).
\end{equation*}

Then
\begin{equation*}
  D_{x_{m}}G_{j}=
  \begin{cases}
    \frac{2\eta_{j}}{\left(1-x_{j}\right)^{2}} & m =j\\
-\frac{\sum_{i\neq j}D_{x_{m}}v^{(j)}_{i} L_{ij}}{z_{j}-\sum_{i\neq j}v^{(j)}_{i} L_{ij}} & m \neq j
  \end{cases}
\end{equation*}

In [ ]:
for s, j in enumerate(B):
    not_j = B[:s] + B[s+1:]
    fail_j = z[j] - np.sum(v[not_j][i] * L[i, j] for i in not_j)
    
    Dx_G[s, s] = (2. * eta_j / (1. - x[1:]) ** 2).reshape(bulge)
    for m in not_j:
        Dx_G[s, axis[m]] = (-np.sum(Dx_v[i, m] * L[i, j] for i in not_j) / fail_j).reshape(pinch)

and
\begin{equation*}
  D_{x_{m}x_{n}}G_{j}=
  \begin{cases}
    \frac{4\eta_{j}}{\left(1-x_{j}\right)^{3}} & m =j=n\\
-\frac{
\sum_{i\neq j}D_{x_{m}x_{n}}v^{(j)}_{i} L_{ij}}{z_{j}-\sum_{i\neq j}v^{(j)}_{i} L_{ij}}
-\frac{\left(\sum_{i\neq j}D_{x_{m}}v^{(j)}_{i}
  L_{ij}\right)
\left( \sum_{i\neq j}D_{x_{n}}v^{(j)}_{i} L_{ij}\right)}
{\left(z_{j}-\sum_{i\neq j}v^{(j)}_{i} L_{ij}\right)^{2}} & m \neq j
\neq n\\
0 &\text{otherwise}.
  \end{cases}
\end{equation*}

In [ ]:
    Dxx_G[j, j, j] = (4. * eta_j / (1. - x[1:]) ** 3).reshape(bulge)
    for m, n in combinations_with_replacement(not_j, 2):
        Dxx_G[j, m, n] = (-np.sum(Dxx_v[i, m, n] * L[i, j] for i in not_j) / fail_j
                          - (np.sum(Dx_v[i, m] * L[i, j] for i in not_j)
                             * np.sum(Dx_v[i, n] * L[i, j] for i in not_j)) / fail_j ** 2).reshape(pinch)

From equation 1 and the fact that
 $D_{\boldsymbol{y}}\boldsymbol{G}=-\boldsymbol{I}$, we have
\begin{equation*}
  \left(D_{\boldsymbol{y}}\boldsymbol{x}\right) = \left( D_{\boldsymbol{x}} \boldsymbol{G}\right)^{-1}
\end{equation*}

In [ ]:
Dx_G_shift = np.moveaxis(Dx_G, (0, 1), (-2, -1))
inv_Dx_G = np.moveaxis(np.linalg.inv(Dx_G_shift), (-2, -1), (0, 1))

def Dy_x(i, k):
    return inv_Dx_G[axis[i], axis[k]]

while from equation 2,
\begin{equation*}
 \left(D_{y_{k}y_{l}}\boldsymbol{x}\right) =
 -\left(D_{\boldsymbol{x}}\boldsymbol{G}\right)^{-1}
\sum_{mn}\left(D_{y_{k}}x_{m}\right)\left(D_{x_{m}x_{n}}\boldsymbol{G}\right) \left(D_{y_{l}}x_{n}\right).
\end{equation*}

In [ ]:
def Dyy_x(i, k, l):
    return -np.sum(
        inv_Dx_G[axis[i], s] * (np.sum(Dy_x(m, k) * Dy_x(m, l) * Dxx_G[j, m, m] for m in B)
                                + np.sum((Dy_x(m, k) * Dy_x(n, l) + Dy_x(n, k) * Dy_x(m, l)) * Dxx_G[j, m, n]
                                         for m, n in combinations(B[:s] + B[s+1:], 2)))
        for s, j in enumerate(B))

We need the mapping between $x$ and $y$ bijective. This is implied if the derivative $D_{\boldsymbol{x}}\boldsymbol{G}$ (= $D_{\boldsymbol{x}}\boldsymbol{y}$) is positive definite everywhere.

In [ ]:
try:
    np.linalg.cholesky(Dx_G_shift + Dx_G_shift.swapaxes(-2, -1))
except np.linalg.LinAlgError:
    print("Dx_G may not be positive definite everywhere.")